### Домашнее задание 1

Задание выполнили студентки БММ181: Желткова Анастасия, Карпулова Алина, Прохожева Диана, Шелепова Юлия

#### Задание: 

Используя train_test_split из sklearn.model_selection разбить датасет на две части: X_test (содержит информацию о пользователе и фильме), y_test (оценка) и аналогичные X_train, y_train. Далее задача предсказать y_train. Метрика качества (стандартная Mean squared error).

Данная задача нацелена на предсказание рейтинга фильмов. 
Тем самым нашей основной метрикой будет RMSE, а не MSE, так как мы сразу видим нашу ошибку исходя из рейтинга. 


Мы очень долго думали,каким же способом было бы легче и удобнее строить модели, для прогнозирования рейтинга фильмов. Изучив различные библиотеки и алгоритмы, мы наткнулись на библиотеку Surprise. Ее расшифровка - Simple Python RecommendatIon System Engine. То есть по сути нам нет нужды самим прописывать и продумывать сложные алгоритмы, когда уже все готово. Зачем придумывать велосипед. Наша главная задача в данной ситуации, была разобраться в документации библиотеки и понять, как ее использовать в наших целях.


После изучения документации, мы остановились на следующих моделях

Модели, которые использовали для предсказания рейтинга:


SVD - Метод сингулярных разложений матриц, один из самых популярных в построении рекомендаций.

SVD++ - Алгоритм SVD ++, расширение SVD с учетом неявных рейтингов.

KNNWithMeans - Основной алгоритм коллаборативной фильтрации, учитывающий средние оценки каждого пользователя.

KNNBasic - Основной алгоритм коллаборативной фильтрации.

KNNBaseline - Основной алгоритм коллаборативной фильтрации с учетом базового рейтинга.

NMF - Алгоритм коллаборативной  фильтрации, основанный на неотрицательной матричной факторизации.

SlopeOne - Простой, но точный алгоритм коллаборативной фильтрации.

KNNWithZScore - Основной алгоритм коллаборативной фильтрации с учетом нормализация z-показателя каждого пользователя.

Более подробную информацию об этих моделях и как они работают можно прочитать в документации библиотеки surprise.

Также мы думали воспользоваться библиотекой reco.recommender, но к сожалению при закгрузке ее через терминал в аканоду выходили ошибки. Но изучив ее, можно сказать, что он могла быть удобнее и лучше, чем библиотека surprise.



Ссылка на библиотеку Surprise - http://surpriselib.com/

Ссылка на reco.recommender - https://github.com/mayukh18/reco

Сперва мы подгружаем часть библиотек, которые нам точно понадобятся и уже походу задания будем добавлять новые.

In [1]:
# импорт необходимых библиотек
from surprise import Reader
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import time
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import GridSearchCV

Затем мы загружем наши данные по информации о пользователях, фильмах и их рейтинг.

In [2]:
ratings = pd.read_csv("ratings.csv")
tags= pd.read_csv("tags.csv")
movies = pd.read_csv("movies.csv")

In [3]:
movies = movies.merge(ratings, on = 'movieId', how = 'inner')

Далее мы создаем датасет, который необходим нам, чтобы предсказывать оценку. Для рекомендательной системы используются 3 переменные- userid , movieid и rating.

In [4]:
df = movies.groupby(by=['userId','movieId'], as_index=False, sort=False).mean()
df = df[['userId','movieId','rating']]
df

,userId,movieId,rating
0,1,1,4.0
1,5,1,4.0
2,7,1,4.5
3,15,1,2.5
4,17,1,4.5
5,18,1,3.5
6,19,1,4.0
7,21,1,3.5
8,27,1,3.0
9,31,1,5.0


Сперва мы бы хотели проверить наивный прогноз, а что если все остальные оценки будут одинаковыми. Насколько сильно будет велика наша ошибка?

In [5]:
y = df["rating"]

df = df.drop(['rating'], axis=1)
features = list(df.columns)
X = df

In [6]:
from sklearn.model_selection import train_test_split
X_train , X_test ,y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
from sklearn.metrics import mean_squared_error as mse
y_mean = np.mean(y_train)                     # посчитали среднее 
y_pred_naive = np.ones(len(y_test)) * y_mean  # спрогнозировали
y_pred_naive[:5]

array([3.5028326, 3.5028326, 3.5028326, 3.5028326, 3.5028326])

In [8]:
import math
rmse = math.sqrt(mse(y_test, y_pred_naive))
rmse

1.04475933055856

Очевидно, наивный прогноз - в данной ситуации это одна из худших моделей. Но теперь мы можем узнать разницу между сложными алгоритмами и обычным наивным прогнозом

Заново загружем данные, так как мы удалили колонку рейтинг из датасета для наивного прогноза

In [9]:
df = movies.groupby(by=['userId','movieId'], as_index=False, sort=False).mean()
df = df[['userId','movieId','rating']]
df.head()

,userId,movieId,rating
0,1,1,4.0
1,5,1,4.0
2,7,1,4.5
3,15,1,2.5
4,17,1,4.5


У библиотеки surprise имеются свои датасеты для работы, но если мы хотим настроить свой, то необходимо ввести ряд параметров и загрузить датасет в тот вид, в котором модели для предсказаний этой библиотеки читали бы данные.

In [10]:
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(df, reader)

Сперва мы просто проверяем модель без разделения на train и test и проводим кросс валидацию. Также мы добавили метрику MAE.

In [11]:
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate

algo = SVD()

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8807  0.8721  0.8766  0.8721  0.8741  0.8752  0.0032  
MAE (testset)     0.6763  0.6693  0.6692  0.6729  0.6729  0.6721  0.0026  
Fit time          5.04    5.00    5.02    5.04    5.00    5.02    0.02    
Test time         0.17    0.17    0.13    0.12    0.17    0.15    0.02    


{'test_rmse': array([0.88074807, 0.87214186, 0.87663364, 0.87214472, 0.87412232]),
 'test_mae': array([0.6763057 , 0.66933093, 0.66923478, 0.67292077, 0.67286319]),
 'fit_time': (5.035577774047852,
  5.000511407852173,
  5.018805503845215,
  5.040560245513916,
  5.002658843994141),
 'test_time': (0.17154383659362793,
  0.17254018783569336,
  0.12783193588256836,
  0.12366938591003418,
  0.16858220100402832)}

Отлично, мы получили результаты, наша средняя RMSE - 0.87

Теперь начнем строить модели используя разбивку на train и test и постараемся улучшить ее.

### SVD

In [13]:
%%time
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split


# используем 25 % данных для тестовой выборки
trainset, testset = train_test_split(data, test_size=.25)

# выбор модели
algo = SVD()

algo.fit(trainset)
predictions = algo.test(testset)

# RMSE
accuracy.rmse(predictions)

RMSE: 0.8736
Wall time: 5.21 s


### SVD++

In [14]:
%%time
from surprise import SVDpp
# используем 25 % данных для тестовой выборки
trainset, testset = train_test_split(data, test_size=.25)

# выбор модели
algo = SVDpp()

algo.fit(trainset)
predictions = algo.test(testset)

# RMSE
accuracy.rmse(predictions)

RMSE: 0.8583
Wall time: 8min 35s


### KNNWithMeans

In [15]:
from surprise import KNNWithMeans

# используем 25 % данных для тестовой выборки
trainset, testset = train_test_split(data, test_size=.25)

# выбор модели
algo = KNNWithMeans()

algo.fit(trainset)
predictions = algo.test(testset)

# RMSE
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9001


0.9000856094605502

### KNNBasic

In [16]:
from surprise import KNNBasic
# используем 25 % данных для тестовой выборки
trainset, testset = train_test_split(data, test_size=.25)

# выбор модели
algo = KNNBasic()

algo.fit(trainset)
predictions = algo.test(testset)

# RMSE
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.9583


0.9582837971278426

### KNNBaseline

In [17]:
from surprise import KNNBaseline

# используем 25 % данных для тестовой выборки
trainset, testset = train_test_split(data, test_size=.25)

# выбор модели
algo = KNNBaseline()

algo.fit(trainset)
predictions = algo.test(testset)

# RMSE
accuracy.rmse(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8770


0.8770488965989934

### NMF

In [18]:
from surprise import NMF
# используем 25 % данных для тестовой выборки
trainset, testset = train_test_split(data, test_size=.25)

# выбор модели
algo = NMF()

algo.fit(trainset)
predictions = algo.test(testset)

# RMSE
accuracy.rmse(predictions)

RMSE: 0.9231


0.9230703013373539

### SlopeOne

In [19]:
from surprise import SlopeOne

# используем 25 % данных для тестовой выборки
trainset, testset = train_test_split(data, test_size=.25)

# выбор модели
algo = SlopeOne()

algo.fit(trainset)
predictions = algo.test(testset)

# RMSE
accuracy.rmse(predictions)

RMSE: 0.9006


0.9005560507751065

### CoClustering

In [20]:
from surprise import CoClustering

# используем 25 % данных для тестовой выборки
trainset, testset = train_test_split(data, test_size=.25)

# выбор модели
algo = CoClustering()

algo.fit(trainset)
predictions = algo.test(testset)

# RMSE
accuracy.rmse(predictions)

RMSE: 0.9442


0.9442174494128958

### KNNWithZScore

In [21]:
from surprise import KNNWithZScore

# используем 25 % данных для тестовой выборки
trainset, testset = train_test_split(data, test_size=.25)

# выбор модели
algo = KNNWithZScore()

algo.fit(trainset)
predictions = algo.test(testset)

# RMSE
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8934


0.8934450503900331

Так как лучшие результаты были у SVD и SVD++ , мы решили попробовать еще метод, чтобы сделать результаты более точными.Для более точной и объективной оценки, мы воспользовались идеей и примером одного из авторов данной библиотеки. Этот скрипт позволяет разделить выборку на 2 части, где A - это выборка, которая применяется для настройки параметров модели, а B для объективной оценки. Тем самым этот скрипт позволит дать более точную и качественную оценку. Но минус скрипта заключается в том, что модель очень долго строится и просчитывается. 

In [22]:
%%time
import random

from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import GridSearchCV

# Загружаем датасет

raw_ratings = data.raw_ratings

# перемешка данных
random.shuffle(raw_ratings)

# A = 80 процентов всех данных, B = 20% всех данных
threshold = int(.8 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # набор А

# Установка параметров для модели через gridsearch
print('Grid Search...')
param_grid = {'n_epochs': [5, 10, 30, 50], 'lr_all': [0.002, 0.005]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=5)
grid_search.fit(data)

algo = grid_search.best_estimator['rmse']

# Обучим на выборке А 
trainset = data.build_full_trainset()
algo.fit(trainset)

# Вычислим смещенную точность на выборке А
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

# Вычислим объективную точность на выборке В
testset = data.construct_testset(B_raw_ratings)  # Набор
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)

Grid Search...
Biased accuracy on A,   RMSE: 0.6328
Unbiased accuracy on B, RMSE: 0.8705
Wall time: 3min 38s


In [23]:
%%time


# A = 80 процентов всех данных, B = 20% всех данных
threshold = int(.8 * len(raw_ratings))
A_raw_ratings = raw_ratings[:threshold]
B_raw_ratings = raw_ratings[threshold:]

data.raw_ratings = A_raw_ratings  # набор А

# Установка параметров для модели через gridsearch
print('Grid Search...')
param_grid = {'n_epochs': [5, 10, 30, 50], 'lr_all': [0.002, 0.005]}
grid_search = GridSearchCV(SVDpp, param_grid, measures=['rmse'], cv=5)
grid_search.fit(data)

algo = grid_search.best_estimator['rmse']

# Обучим на выборке А 
trainset = data.build_full_trainset()
algo.fit(trainset)

# Вычислим смещенную точность на выборке А
predictions = algo.test(trainset.build_testset())
print('Biased accuracy on A,', end='   ')
accuracy.rmse(predictions)

# Вычислим объективную точность на выборке В
testset = data.construct_testset(B_raw_ratings)  # Набор
predictions = algo.test(testset)
print('Unbiased accuracy on B,', end=' ')
accuracy.rmse(predictions)

Grid Search...
Biased accuracy on A,   RMSE: 0.7345
Unbiased accuracy on B, RMSE: 0.8591
Wall time: 4h 49min 59s


Проведя предсказания используя ряд моделей, мы выявили, что SVD модели лучше всего подходят для нашей подвыборки и лучше всего предсказывают.

Результаты без применения GridsearchCV с разибением на train и test split:

SVD = 0.8736

SVD ++  = 0.8583

C применением GridsearchCV и разбиением на 2 подвыборки А и В.

SVD

Смещенная точность (на выборке A) - 0.6328

Объективная точность (на выборке В) - 0.8705

SVD++

Смещенная точность (на выборке A) - 0.7345

Объективная точность (на выборке В) - 0.8591

И также мы предпологаем, что нашим основным критерием был MSE, а не RMSE. Теперь найдем MSE нашей лучшей модели

RMSE = MSE ^ 1/2

MSE = RMSE ^ 2



In [24]:
MSE_R = 0.8591 ** 2
MSE_R

0.73805281